# Bootcamp SQL

### [Excalidraw](https://app.excalidraw.com/l/8pvW6zbNUnD/3ktnOgfFeRK)
### [github](https://github.com/lvgalvao/data-engineering-roadmap/tree/main/Bootcamp%20-%20SQL%20e%20Analytics)


## Aula 1

Por que Postgres?
- opensource - de graça
- presente em várias empresas
- um dos principais bancos na logica cliente-servidor
- suporta tipos complexos como json

consultar a documentação (link na aula 1 readme)

Comandos principais
- SELECT
  - SELECT colunas ou * FROM tabela
  - SELECT distinct coluna FROM tabela
  - SELECT count(distinct coluna) FROM tabela
- WHERE
  - SELECT * FROM tabela WHERE coluna = 'valor' AND coluna2 = 'valor2'
  - SELECT * FROM tabela WHERE coluna <> 'valor'
  - SELECT * FROM tabela WHERE coluna = 'valor' AND (coluna2 = 'valor2' OR coluna2 = 'valor3')
- ORDER BY
  - SELECT * FROM tabela ORDER BY coluna1 ASC, coluna2 DESC
  - SELECT * FROM tabela ORDER BY coluna1, coluna2
- LIKE e IN (é pesado, poderia concatenar varias colunas em um e dar like nessa coluna quando queremos multiplos likes. O ILIKE resolve o case senstivie to LIKE)
  - SELECT * FROM tabela WHERE coluna LIKE 'a%'
  - SELECT * FROM tabela WHERE coluna NOT LIKE 'a%'
  - SELECT * FROM customers WHERE coluna IN ('valor1', 'valor2', 'valor3')
  - SELECT * FROM customers WHERE coluna LIKE '%or%'
  - SELECT * FROM customers WHERE coluna LIKE '%_r%' (r na segunda posição, '_' significa qualquer caractere)
  - SELECT * FROM customers WHERE coluna LIKE 'a%o' (começa com 'a' e termina com 'o')
  - SELECT * FROM customers WHERE coluna LIKE 'a__%' (começa com 'a' e pelo menos 3 caracteres)
- MAIS AVANÇADOS (gostinho de subquery)
  - SELECT * FROM customers WHERE coluna IN (SELECT DISTINCT coluna from tabela)
  - SELECT * FROM customers WHERE coluna BETWEEN 10 AND 20 (not NOT BETWEEN)
  - SELECT TO_CHAR(data, 'DD-MM-YYY') FROM tabela WHERE order_date between '1996-04-07' and '1996-09-07' (converte data em outro formato)
- Agregadores
  - MIN, MAX, AVG, COUNT, SUM - normalmente usadas com GROUP BY
  - LIMIT # (limita a qtde de resultados)

## Aula 2

Podemos ir no PGAdmin e dentro do banco na parte de Login/Group Roles podemos definir permissões de acesso

- SELECT é usado para consultar dados
- INSERT, UPDATE E DELETE normalmente sao mais usados por engenheiros de dados e principalmente sistemas com esse tipo de acesso (por exemplo CRUD)
- CREATE, ALTER, DROP sao mais usados por DBA e modeladores do banco
- GRANT, REVOKE tem a ver com acesso, usado por admin do banco - tira, dá ou deleta conta
- BEGIN, COMMIT, ROLLBACK tem a ver com transações. porém transações sempre sao um conjunto de comandos. Porém para garantir a atomicidade, colocamos os statements entre um BEGIN e um COMMIT para garantir que tudo que está entre eles rode. Caso algo dê errado no entre BEGIN e COMMIT, a transação nao acontece. Se der um erro eu posso usar o ROLLBACK para voltar, como se fosse um try/except do python

Sempre trazer as colunas que precisamos, evitar usar o SELECT * pq pressiona a performance


# Aula 3

importante a sequencia de comandos em SQL:
1. select
2. from
3. join
4. where
5. group by
6. having
7. order by
8. limit

## Joins

- Inner (padrão)
- left 
- right
- full
- cross (menos utilizada)

Se estamos usando funções agregadoras no select, o que nao está sendo agregado precisa estar no group by

# Aula 4 - Window functions

Como se criasse nova coluna, similar a usar um dataframe no pandas

windows function permite atingir o mesmo resultado com statements diferentes. Exemplo do Northwind:

```sql
SELECT order_id,
       COUNT(order_id) AS unique_product,
       SUM(quantity) AS total_quantity,
       SUM(unit_price * quantity) AS total_price
FROM order_details
GROUP BY order_id
ORDER BY order_id;

-- é equivalente a 

SELECT DISTINCT order_id,
   COUNT(order_id) OVER (PARTITION BY order_id) AS unique_product,
   SUM(quantity) OVER (PARTITION BY order_id) AS total_quantity,
   SUM(unit_price * quantity) OVER (PARTITION BY order_id) AS total_price
FROM order_details
ORDER BY order_id;
```

Windows function basicamente substitui o group by porém é feito ao nível de linha e o OVER + PARTITION BY deve ser incorporado a todas as colunas selecionadas. 

POdemos inclusive fazer o partition by para colunas diferentes gerando agrupamentos diferentes por linha

É possível ter mais de uma coluna em um partition by, o que pode ser útil para algumas análises

A boa prática é já trazer o dado do SQL do jeito que vc precisa, vc pode até trazer o dado bruto e tratar em outro local como um pandas, porém estará consumindo mais memória. Ex: 
- se vc joga um select * from e manda pro powerbi vai ficar lento e vai ter q ter cálculo no pbi, que não é o ideal.
- O mesmo vale para regra de negócio colocada no front end que não é o ideal, como calcular um desconto no frontend ao invés de ter isso já calculado no banco

Em termos de performance podemos usar EXPLAIN ou EXPLAIN ANALYZE antes da query para ver a performance

RANK, DENSE e ROWNUMBER servem para fazer ordenação com base em alguma coisa:
- ROWNUMBER nao tem repetido, ele dá um jeito de desempatar, usa uma coluna ID implícita para ordenar, é não-determinístico em caso de empate
- RANK tem repetido, nao desempata e ordena normalmente, entao se tem 2 em 1o, o próximo será 3o         
- DENSE tem repetido, nao desempata e ordena pulando, entao se tem 2 em 1o, o próximo será 2o         

PERCENT_RANK() e CUME_DIST() sao menos utilizados mas podem ser úteis quando queremos ver % dentro de grupos

NTILE() é usada para dividir o conjunto de resultados em um número especificado de partes aproximadamente iguais ou "faixas" e atribuir um número de grupo ou "bucket" a cada linha com base em sua posição dentro do conjunto de resultados ordenado.

LAG() e LEAD() serve para comparar e ver evoluções / variações

# Exercícios Aula 4 (chat-gpt/Northwind)

Group 1: RANK, DENSE_RANK, ROW_NUMBER
Exercises:
1. Rank orders by their total value (descending) for each customer. Include ties.

```sql
SELECT
	c.company_name,
	o.order_id,
	SUM(od.unit_price * od.quantity) AS order_total,
	RANK() OVER(PARTITION BY c.company_name ORDER BY SUM(od.unit_price * od.quantity) DESC) AS order_rank
FROM
	customers c
JOIN
	orders o on c.customer_id = o.customer_id
JOIN
	order_details od on o.order_id = od.order_id
GROUP BY
	c.company_name, o.order_id, od.unit_price, od.quantity
```


2. Rank products (DENSE_RANK) based on the number of orders placed, grouped by category.

```sql
SELECT
	p.product_name,
	c.category_name as category,
	COUNT(od.order_id) AS order_qty,
	DENSE_RANK() OVER(PARTITION BY c.category_name ORDER BY COUNT(od.order_id) DESC) AS order_dense
FROM
	products p
JOIN
	categories c on p.category_id = c.category_id
JOIN
	order_details od on p.product_id = od.product_id
GROUP BY
	p.product_name, c.category_name
```

3. Assign a row number (ROW_NUMBER) to each employee within their respective city, ordered by their hire date.
4. Rank employees by their sales amount. Use RANK to ensure ties are ranked the same and leave gaps in ranks.
5. Assign ROW_NUMBER to products within each supplier, ordered by price.


Group 2: PERCENT_RANK, CUME_DIST
Exercises:
1. Calculate the PERCENT_RANK of each order based on its total value, grouped by customer.
2. Compute the PERCENT_RANK of employees based on their hire date (earliest hire date = 0, latest = 1).
3. Calculate the CUME_DIST of orders based on their total value across all orders (not grouped).
4. Determine the CUME_DIST of products based on their price, grouped by category.
5. For each employee, compute the PERCENT_RANK of their total sales.

Group 3: NTILE
Exercises:
1. Divide all orders into 4 equal-sized groups (quartiles) based on their total value.
2. Divide employees into 5 NTILE groups based on their hire date, with the earliest hire date in NTILE 1.
3. Categorize products into 3 NTILE groups within each category, based on their unit price.
4. Partition orders by year and assign each order to 4 NTILE groups based on their total value within that year.
5. Divide customers into 10 NTILE groups based on the total number of orders placed by each.

Group 4: LAG, LEAD
Exercises:
1. For each order, use LAG to calculate the time difference (in days) between the current order and the previous order for the same customer.
2. For each product, use LEAD to display the unit price of the next product within the same category.
3. For each employee, compute their sales amount and show the sales amount of the previous employee using LAG.
4. For each order, display the next order's total value for the same customer using LEAD.
5. For each product, calculate the difference between its unit price and the unit price of the next product in the same category using LEAD.